# Textmod Pipeline Processing

In [1]:
# prep environment
import pandas as pd
import numpy as np

In [2]:
# load pipeline export 
pipeline_export = pd.read_csv('/Users/smarshall/Desktop/threat_pipeline_export.csv')

In [53]:
import json

test = '{"binary_result":{"id":1177441622,"pipeline_task_id":801459368,"project_id":42796,"status_format":"category","task_id":"202fea10-d99c-11ed-8326-772e658dd403","focus":null,"media_objects":null,"known_status":null,"status_to_edit":null,"state":"finished","status":"no","created_on":"2023-04-13T01:40:09.645Z","finished_on":"2023-04-14T18:56:14.443Z","invalidated_on":null,"text_data":"ahaha hope her favourite toy goes missing and when she finds it i hope it‚Äôs head is ripped off","task_description":null,"remote_status":null,"overlay_images":null,"moderator_blacklist":null,"task_description_translations":null,"hsl_options":null,"text":null,"media":null},"regular_result":null}'

# json_test = json.loads(test)

# type(json_test)

type(test)

str

## Format Pipeline Export

Extract Statuses

In [6]:
# attach binary_result column
pipeline_export['binary_result'] = pipeline_export['status'].str.split('"status"').str[1].str.split('"').str[1]

# attach multi_result column
pipeline_export['multi_result'] = np.where((pipeline_export['binary_result'] == 'no'), 
                                            'benign', 
                                            pipeline_export['status'].str.split('"status"').str[2].str.split('"').str[1])

# pipeline_export.head()

Display Summary Stats

In [7]:
# display summary stats on pipeline job
print("Tasks pending in multi-cat job:", len(pipeline_export.loc[pipeline_export['multi_result'].isna()]))

# generate frequency table
pipeline_export.multi_result.value_counts()

Tasks pending in multi-cat job: 1156


benign              6756
controversial       5717
incitement          3701
threat               622
foreign_language      48
Name: multi_result, dtype: int64

Write Formatted Pipeline Export

In [6]:
# export formatted pipeline data
pipeline_export.to_csv('/Users/smarshall/Desktop/threat_pipeline_export_formatted.csv', index=False)

## Generate QA Subsets

### Generate Binary QA Subset

In [ ]:
# set audit size 
n_audit = 2000

# subset to benign tasks
benign = pipeline_export.loc[pipeline_export['binary_result'] == 'no']

# select random sample
benign_qa = benign.sample(n_audit)[['text_data', 'multi_result']]

# preview
benign_qa

# export binary QA csv
benign_qa.to_csv('/Users/smarshall/Desktop/benign_qa.csv', index=False)

### Generate Multi-Class Subset

In [1]:
import pandas as pd


pipeline_export = pd.read_csv('/Users/smarshall/Desktop/threat_export.csv')

In [2]:
# generate class distribution
pipeline_export.multi_result.value_counts()

["controversial"]       5417
["incitement"]          3744
["benign"]              2931
["inconclusive"]        2297
["threat"]               907
["foreign_language"]      50
Name: multi_result, dtype: int64

In [4]:
# set number of tasks per class
n_threat = 650
n_incitement = 300
n_controversial = 400
n_benign = 650

# threat
threat_sample = pipeline_export.loc[pipeline_export['multi_result'] == '["threat"]'].sample(n = n_threat)

# incitement
incitement_sample = pipeline_export.loc[pipeline_export['multi_result'] == '["incitement"]'].sample(n = n_incitement)

# controversial
controversial_sample = pipeline_export.loc[pipeline_export['multi_result'] == '["controversial"]'].sample(n = n_controversial)

# benign
benign_sample = pipeline_export.loc[pipeline_export['multi_result'] == '["benign"]'].sample(n = n_benign)

# stack all together
stacked_samples = pd.concat([threat_sample, incitement_sample, controversial_sample, benign_sample], ignore_index=True, axis=0)

# subset table
multi_qa = stacked_samples[['text_data', 'multi_result']]

# preview
multi_qa

# export multi QA csv
multi_qa.to_csv('/Users/smarshall/Desktop/multi_qa.csv', index=False)